<a href="https://colab.research.google.com/github/greyhound101/Data_Science_bowl-2019/blob/master/DSB-2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle competitions download -c data-science-bowl-2019

 98% 40.0M/40.8M [00:00<00:00, 52.2MB/s]
100% 40.8M/40.8M [00:00<00:00, 92.1MB/s]
  0% 0.00/399k [00:00<?, ?B/s]
100% 399k/399k [00:00<00:00, 125MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 11.4MB/s]
100% 397M/397M [00:03<00:00, 151MB/s]

  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 83.9MB/s]


In [4]:
import pandas as pd
import numpy as np
def read_data():
    train = pd.read_csv('train.csv.zip',nrows=100000)
    
    test = pd.read_csv('test.csv.zip')
    
    train_labels = pd.read_csv('train_labels.csv.zip',nrows=100000)
    
    specs = pd.read_csv('specs.csv',nrows=100000)

    sample_submission = pd.read_csv('sample_submission.csv',nrows=100000)
    return train, test, train_labels, specs, sample_submission

def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = train['title'].astype(str)+train['event_code'].astype(str)
    test['title_event_code'] = test['title'].astype(str)+test['event_code'].astype(str)
    
    titles = list(set(train['title'].unique()).union(set(test['title'].unique())))
    event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    title_code = list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique())))
    
    
    
    event_code_map = dict(zip(event_code, np.arange(len(event_code))))
    
    title_map = dict(zip(titles, np.arange(len(titles))))
    title_demap = dict(zip(np.arange(len(titles)),titles ))
    
    event_id_map = dict(zip(event_id,np.arange(len(event_id))))
    world_map = dict(zip(worlds, np.arange(len(worlds))))
    
    title_code_map=dict(zip(title_code,np.arange(len(title_code))))
    title_code_demap=dict(zip(np.arange(len(title_code)),title_code))
    
    
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(title_map)
    test['title'] = test['title'].map(title_map)
    train['world'] = train['world'].map(world_map)
    test['world'] = test['world'].map(world_map)
    train_labels['title'] = train_labels['title'].map(title_map)
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    
    win_code={col:4100 for col in train.loc[train['type']=='Assessment']['title'].unique()}
    win_code[title_map['Bird Measurer (Assessment)']]=4110
    
    return train, test, train_labels, title_map,title_demap,world_map,title_code,win_code,event_code_map,event_id_map

train, test, train_labels, specs, sample_submission = read_data()
train, test, train_labels, title_map,title_demap,world_map,title_code,win_code,event_code_map,event_id_map = encode_title(train, test, train_labels)

def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_val=[]
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False))):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False)):
        test_data,val_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
        compiled_val+=(val_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_val = pd.DataFrame(compiled_val)
    categoricals = ['session_title']
    return reduce_train, reduce_test,reduce_val

from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
%matplotlib inline
import time
from collections import Counter
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb

def get_data(user_sample, test_set=False):
    all_assessments=[]
    event_code_count: Dict[str, int] = {ev: 0 for ev in list(event_code_map.keys())}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list(event_id_map.keys())}
    title_count: Dict[str, int] = {eve: 0 for eve in list(title_map.keys())} 
    title_code_count: Dict[str, int] = {t_eve: 0 for t_eve in title_code}
    
    
    
    ass_try_non_rec=0
    ass_true_non_rec=0
    ass_false_non_rec=0


    ass_try_rec=0
    ass_true_rec=0
    ass_false_rec=0
  
  
    try_non_rec=0
    true_non_rec=0
    false_non_rec=0

    cnt=0
    try_rec=0
    true_rec=0
    false_rec=0
    cum_acc=0
    for i, session in user_sample.groupby('game_session', sort=False):
        
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = title_demap[session_title]
                    
        rec=session.loc[session['event_code'].isin([4100,4110])]
        non_rec=session.loc[~(session['event_code'].isin([4100,4110]))]
        
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            features={}
            all_attempts = session.loc[session['event_code']==win_code[session_title]]
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_code_count.copy())
            
            
            #recorded_assessments
            features['true_rec_ass']=ass_true_rec
            features['false_rec_ass']=ass_false_rec
            features['tot_rec_ass']=ass_try_rec
    
            #non_recorded assessmnents
            features['true_non_rec_ass']=ass_true_non_rec
            features['false_non_rec_ass']=ass_false_non_rec
            features['tot_non_rec_ass']=ass_try_non_rec
    
            #all recorded
            features['true_rec_non_ass']=true_rec
            features['false_rec_non_ass']=false_rec
            features['tot_rec_non_ass']=try_rec
    
            #all non recorded
            features['true_non_rec']=true_non_rec
            features['false_non_rec']=false_non_rec
            features['tot_non_rec']=try_non_rec
            if cnt!=0:
                features['accuracy_cum']=cum_acc/cnt
            else:
                features['accuracy_cum']=0
            cnt+=1
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session['title'].iloc[0]
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            cum_acc+=accuracy
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            true=rec.event_data.str.contains('true').sum()
            false=rec.event_data.str.contains('false').sum()
            tot=true+false
        
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                
                ass_true_rec+=true
                ass_false_rec+=false
                ass_try_rec=ass_false_rec+ass_true_rec
                
                
                all_assessments.append(features)
      
            else:
            #non rec ass
                ass_true_non_rec+=non_rec.event_data.str.contains('true').sum()
                ass_false_non_rec+=non_rec.event_data.str.contains('false').sum()
                ass_tot_non_rec=ass_true_non_rec+ass_false_non_rec
      
    
    
            
        
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = title_demap[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_code_count = update_counters(title_code_count, 'title_event_code')
    
        #non rec
        true_non_rec+=(non_rec['event_data'].str.contains('true')).sum()
        false_non_rec+=(non_rec['event_data'].str.contains('false')).sum()
        tot_non_rec=true_non_rec+false_non_rec
        #rec
        true_rec+=(rec['event_data'].str.contains('true')).sum()
        false_rec+=(rec['event_data'].str.contains('false')).sum()
        tot_rec=false_rec+true_rec
        # counts how many actions the player has done so far, used in the feature of the same name
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1],all_assessments[:-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments
reduce_train, reduce_test, reduce_val= get_train_and_test(train, test)
reduce_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_test.columns]
reduce_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_train.columns]

157it [00:11, 13.44it/s]
100%|██████████| 1000/1000 [01:57<00:00,  8.48it/s]


In [5]:

trn_ft=lgb.Dataset(reduce_train.select_dtypes(exclude=object).drop(['accuracy_group'],1),reduce_train['accuracy_group'])
val_ft=lgb.Dataset(reduce_val.select_dtypes(exclude=object).drop(['accuracy_group'],1),reduce_val['accuracy_group'])
mod=lgb.train( {'feature_fraction': 0.7975023410800904,'n_estimators':7297,'learning_rate':0.001,
  'max_depth': 14,
  'subsample': 0.7531064105216918},trn_ft)
pre=mod.predict(reduce_test.select_dtypes(exclude=object).drop(['accuracy_group'],1))
x,y,z=1.21519937, 1.70523477, 2.25945307
res=[]
for i in pre:
        if i<x:
            res.append(0)
        elif i<y:
            res.append(1)
        elif i<z:
            res.append(2)
        else:
            res.append(3)


sample_submission['accuracy_group']=res
sample_submission.to_csv('submission.csv', index=False)
sample_submission['accuracy_group'].value_counts()

3    447
2    198
0    192
1    163
Name: accuracy_group, dtype: int64